In [37]:
import pandas as pd
import geopandas
import datetime
from geopandas import GeoDataFrame
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [18]:
#import fire calls into pandas
fireCalls = pd.read_csv("Seattle_Real_Time_Fire_911_Calls.csv")

#remove unnecessary initial line
fireCalls = fireCalls.drop([0, 0])

#add columns to break down data further
#https://erikrood.com/Python_References/extract_month_year_pandas_final.html
fireCalls['Month'] = pd.DatetimeIndex(fireCalls['Datetime']).month
fireCalls['Year'] = pd.DatetimeIndex(fireCalls['Datetime']).year

In [21]:
#separate 2017 and 2018 record
fc2017 = fireCalls[fireCalls.Year == 2017]
fc2018 = fireCalls[fireCalls.Year == 2018]

In [22]:
#import seattle neighborhood shapefile
seaNH = geopandas.read_file("City_Clerk_Neighborhoods\City_Clerk_Neighborhoods.shp")

#need to remove excess rows at beginning for extra islands
seaNH = seaNH.drop(seaNH.index[:26])

In [31]:
#get seaNH projection data to use for fire calls geodata frame 
seaNH_crs = seaNH.crs

#convert fireCalls to geodataframe
#https://gis.stackexchange.com/questions/174159/convert-a-pandas-dataframe-to-a-geodataframe
geometry1 = [Point(xy) for xy in zip(fc2017.Longitude, fc2017.Latitude)]
crs = seaNH_crs
fc2017_geo = GeoDataFrame(fc2017, crs = crs, geometry = geometry1)

geometry2 = [Point(xy) for xy in zip(fc2018.Longitude, fc2018.Latitude)]
crs = seaNH_crs
fc2018_geo = GeoDataFrame(fc2018, crs = crs, geometry = geometry2)


In [32]:
#spatial join fc_geo to neighborhoods so each fire call has a neighborhood attribute
fc2017_seattle = geopandas.sjoin(fc2017_geo, seaNH, how = 'inner', op = 'within')
fc2018_seattle = geopandas.sjoin(fc2018_geo, seaNH, how = 'inner', op = 'within')

In [35]:
fc2017_seattle.to_file("2017_Fire_Calls_Seattle")
fc2018_seattle.to_file("2018_Fire_Calls_Seattle")